In [ ]:
from confluent_kafka import Consumer, Producer, KafkaError
from fastavro import writer, reader, parse_schema, schemaless_reader
import json
import io

with open('/home/ubuntu/divolte-collector-0.9.0/conf/MyEventRecord0.avsc') as f:
    schema = parse_schema(json.loads(f.read()))

def decode(msg_value):
    message_bytes = io.BytesIO(msg_value)
    return schemaless_reader(message_bytes, schema)

def getOkk(lst):
    ret ={}
    for x,y in lst.items():
        if x in ['sessionId', 'timestamp', 'location', 'id_product']:
            if x == 'id_product':
                if y != None:
                    ret[x] = y
            else:
                ret[x] = y
                if x == 'location':
                    ret['id_item'] = ''
                    z = y.split('/')
                    if len(z) > 3:
                        n = (-2 if z[-1] == '' else -1)
                        if z[n-1] == 'item':
                            ret['id_item'] = z[n]
        elif x in ['price_product','total_price_product']:
            z = y if y == None else float(y[:-5].replace(' ',''))
            if z != None:
                ret[x] = z
    if ret.get('id_product','') > '':
        ret['item_count'] = (None if ret['price_product'] == None else
                             int(ret['total_price_product']/ret['price_product']))
    return ret

ip, topic = '35.195.166.173:6667', 'andrey.selivanov'
topicTo1, topicTo2 = 'users_json', 'orders_json'
conf = {'bootstrap.servers': ip}
p1 = Producer(**conf)
p2 = Producer(**conf)
c = Consumer(
    {'bootstrap.servers': ip,
     'group.id': 'avro2json_group',
     'auto.offset.reset': 'latest',
     "enable.auto.commit": True})
c.subscribe([topic])

running = True
while running:
    msg = c.poll()
    if not msg.error():
        msg_value = msg.value()
        print(msg_value)
        event_dict = getOkk(decode(msg_value))
        if event_dict.get('id_product','') > '':
            p2.produce(topicTo2, json.dumps(event_dict))
        else:
            p1.produce(topicTo1, json.dumps(event_dict))
        print(event_dict)
    elif msg.error().code() != KafkaError._PARTITION_EOF:
        print(msg.error())
        running = False

b'V0:jnytq3fk:hltn4amRywpScAQcEN6eq7dEQ9OSQrfA\xfe\xc6\x80\x83\xdaYBhttps://b24-dy0v8s.bitrix24.shop/\x1879.133.79.58\x02\x10pageView\x02vhttps://b24-dy0v8s.bitrix24.shop/personalnyyrazdel/korzina/\x00\x02\xf2\x01Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36\x02"Personal computer\x02\x08OS X\x02\x0e10.13.6\x00\x00\x00'
{'sessionId': '0:jnytq3fk:hltn4amRywpScAQcEN6eq7dEQ9OSQrfA', 'timestamp': 1541091103167, 'location': 'https://b24-dy0v8s.bitrix24.shop/personalnyyrazdel/korzina/', 'id_item': ''}
b'V0:jnytq3fk:hltn4amRywpScAQcEN6eq7dEQ9OSQrfA\x9e\xd9\x80\x83\xdaYvhttps://b24-dy0v8s.bitrix24.shop/personalnyyrazdel/korzina/\x1879.133.79.58\x02\x10pageView\x02Bhttps://b24-dy0v8s.bitrix24.shop/\x00\x02\xf2\x01Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36\x02"Personal computer\x02\x08OS X\x02\x0e10.13.6\x00\x00\x00'
{'sessionId': '0:jnytq3fk:h

from kafka import KafkaConsumer
import codecs
from tqdm import tqdm
import json
# To consume latest messages and auto-commit offsets
ip, topic = '35.195.166.173:6667', 'andrey.selivanov'
consumer = KafkaConsumer(topic,
                         #group_id='my-group',
                         bootstrap_servers=[ip],
                         auto_offset_reset=OffsetType.LATEST,#'earliest',
                         enable_auto_commit=True,
                         consumer_timeout_ms=5000)
for message in tqdm(consumer):
    print(message.value)